In [206]:
import subprocess
import multiprocessing
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import time
from scipy.stats import ttest_rel

In [50]:
neighbourhoods = ["--transpose", "--exchange", "--insert", "--tei", "--tie"]
def run_fssp(instance, init=0, improvement=0, neighbourhood=0):
    try:
        init = "--random-init" if not init else "--srz"
        improvement = "--first" if not improvement else "--best"
        neighbourhood = neighbourhoods[neighbourhood]
        command = "./fssp " + " ".join([instance, init, improvement, neighbourhood])
        beg_time = time.time()
        output = subprocess.check_output(command, shell=True, stderr=subprocess.DEVNULL)
        solution, score = output.decode('utf-8').split('\n')[:2]
        return command, [int(x) for x in solution.split(" ") if x], int(score), time.time() - beg_time
    except Exception as e:
        print(e)
        return command, [], 0


In [51]:
def run_instance(instance):
    res = []
    for init in [0, 1]:
        for improvement in [0, 1]:
            for neighbourhood in [0, 1, 2]:
                res.append(run_fssp(instance, init, improvement, neighbourhood))
        for neighbourhood in [3, 4]:
            res.append(run_fssp(instance, init, 0, neighbourhood))
    return res


In [52]:
def all_instances_names():
    for filename in os.listdir("../instances"):
        if filename != "bestSolutions.txt":
            yield os.path.join("../instances/", filename)

In [53]:
instances = list(all_instances_names())
pool = multiprocessing.Pool(multiprocessing.cpu_count())
res = list(tqdm(pool.imap_unordered(run_instance, instances), total=len(instances)))        

In [57]:
import itertools
a = list(itertools.chain(*res))
len(a)

960

In [58]:
def to_nikita(a):
    times = {}
    for x in a:
        times[x[0].replace("../", "").replace("--tie", "--vnd-tie").replace("--tei", "--vnd-tei")] = x[2]
    return times

times = to_nikita(a)
print(times)

{'./fssp instances/50_20_28 --random-init --first --transpose': 725848, './fssp instances/50_20_28 --random-init --first --exchange': 566459, './fssp instances/50_20_28 --random-init --first --insert': 575218, './fssp instances/50_20_28 --random-init --best --transpose': 724189, './fssp instances/50_20_28 --random-init --best --exchange': 572886, './fssp instances/50_20_28 --random-init --best --insert': 583491, './fssp instances/50_20_28 --random-init --first --vnd-tei': 566488, './fssp instances/50_20_28 --random-init --first --vnd-tie': 559159, './fssp instances/50_20_28 --srz --first --transpose': 565683, './fssp instances/50_20_28 --srz --first --exchange': 565197, './fssp instances/50_20_28 --srz --first --insert': 561436, './fssp instances/50_20_28 --srz --best --transpose': 565683, './fssp instances/50_20_28 --srz --best --exchange': 565197, './fssp instances/50_20_28 --srz --best --insert': 561378, './fssp instances/50_20_28 --srz --first --vnd-tei': 559822, './fssp instances/

In [287]:
processed = [
    {
        "instance": s.split()[1].split("/")[-1],
        "n_jobs": len(sol),
        "init": "srz" if "srz" in s else "random",
        "pivot": "best" if "best" in s else "first",
        "neighbourhood": "exchange" if "exchange" in s else "transpose" if "transpose" in s else "insert" if "insert"in s else "tie" if "tie" in s else "tei",
        "score": score,
        "time": time
    }
    for s, sol, score, time in a
]
df = pd.DataFrame(processed)

In [288]:
df

,init,instance,n_jobs,neighbourhood,pivot,score,time
0,random,50_20_28,50,transpose,first,725848,0.032851
1,random,50_20_28,50,exchange,first,566459,0.777335
2,random,50_20_28,50,insert,first,575218,0.748541
3,random,50_20_28,50,transpose,best,724189,0.016455
4,random,50_20_28,50,exchange,best,572886,0.182460
5,random,50_20_28,50,insert,best,583491,0.260463
6,random,50_20_28,50,tei,first,566488,0.355872
7,random,50_20_28,50,tie,first,559159,0.602742
8,srz,50_20_28,50,transpose,first,565683,0.013924
9,srz,50_20_28,50,exchange,first,565197,0.047436


In [290]:
best = pd.read_csv("../instances/bestSolutions.txt")
best.columns = ["instance", "best_solution"]
best.instance = best.instance.str.strip()
best.best_solution = best.best_solution.astype(int)
best.sample()

,instance,best_solution
7,100_20_08,1754470


In [291]:
merged = pd.merge(df, best, how='left', on='instance')
merged.sample()

,init,instance,n_jobs,neighbourhood,pivot,score,time,best_solution
272,random,50_20_16,50,transpose,first,734590,0.023479,551243


In [292]:
merged['percentage_deviation'] = 100 * (merged.score - merged.best_solution) / merged.best_solution

In [293]:
exercise1 = merged[(merged.neighbourhood != "tei") & (merged.neighbourhood != "tie")]
print(len(exercise1))
exercise1.sample()

720


,init,instance,n_jobs,neighbourhood,pivot,score,time,best_solution,percentage_deviation
649,srz,50_20_10,50,exchange,first,613450,0.059195,600785,2.108075


In [294]:
exercise1_groupby_algos = exercise1.groupby(['init', 'pivot', 'neighbourhood', 'n_jobs'])
exercise1_groupby_algos.mean()[['percentage_deviation']]

percentage_deviation
init   pivot neighbourhood n_jobs                      
random best  exchange      50                  4.385601
                           100                 4.667283
             insert        50                  8.170156
                           100                10.903816
             transpose     50                 32.736274
                           100                42.109242
       first exchange      50                  1.965092
                           100                 1.715059
             insert        50                  6.265681
                           100                 7.724756
             transpose     50                 31.101046
                           100                41.205408
srz    best  exchange      50                  2.787518
                           100                 3.432429
             insert        50                  2.811639
                           100                 3.511336
             transpose     50                  3.701844
                           100                 4.587164
       first exchange      50                  2.858113
                           100                 3.119183
             insert        50                  2.669091
                           100                 3.202606
             transpose     50                  3.680069
                           100                 4.580643

In [270]:
exercise1_groupby_algos.sum()[['time']]

time
init   pivot neighbourhood            
random best  exchange       117.326819
             insert         137.103702
             transpose        2.994078
       first exchange       524.671305
             insert         679.141816
             transpose        3.355781
srz    best  exchange        26.367852
             insert          26.074299
             transpose        1.777759
       first exchange        36.512201
             insert          37.759094
             transpose        1.749614

In [280]:
algos = pd.DataFrame(exercise1_groupby_algos.groups).T
algos = algos.applymap(lambda x: exercise1.loc[x].percentage_deviation)
algos.index = algos.index.tolist()
algos.columns = exercise1.instance.unique()
algos.sample()

,50_20_28,50_20_30,50_20_02,50_20_24,50_20_05,50_20_04,50_20_01,100_20_20,50_20_09,100_20_22,...,50_20_12,100_20_29,100_20_12,50_20_17,50_20_26,100_20_19,100_20_04,100_20_14,100_20_01,100_20_08
"(random, first, exchange)",3.028324,1.743092,2.504571,1.851681,2.436566,1.500103,2.138225,1.082547,0.923538,2.332778,...,1.575491,1.522666,0.324371,1.612121,1.361995,2.348437,1.956799,1.078422,1.349248,1.791253


In [281]:
def df_crossjoin(df1, df2, **kwargs):
    if df1 is df2:
        df2 = df1.copy()
    df1['_tmpkey'] = 1
    df2['_tmpkey'] = 1

    res = pd.merge(df1, df2, on='_tmpkey', **kwargs).drop('_tmpkey', axis=1)
    res.index = pd.MultiIndex.from_product((df1.index, df2.index))

    df1.drop('_tmpkey', axis=1, inplace=True)
    df2.drop('_tmpkey', axis=1, inplace=True)

    return res

algos = df_crossjoin(algos, algos)
algos.shape

(144, 120)

In [282]:
student_ttest_rel = algos.apply(lambda x: ttest_rel(x[:60], x[60:])[1], axis=1)
student_ttest_rel = student_ttest_rel.unstack()
student_ttest_rel > 0.05

/home/toon/Projects/heuristics/impl1/ve/lib/python3.6/site-packages/scipy/stats/stats.py:3851: RuntimeWarning: invalid value encountered in absolute
  prob = distributions.t.sf(np.abs(t), df) * 2  # use np.abs to get upper tail


,"(random, best, exchange)","(random, best, insert)","(random, best, transpose)","(random, first, exchange)","(random, first, insert)","(random, first, transpose)","(srz, best, exchange)","(srz, best, insert)","(srz, best, transpose)","(srz, first, exchange)","(srz, first, insert)","(srz, first, transpose)"
"(random, best, exchange)",False,False,False,False,False,False,False,False,False,False,False,False
"(random, best, insert)",False,False,False,False,False,False,False,False,False,False,False,False
"(random, best, transpose)",False,False,False,False,False,False,False,False,False,False,False,False
"(random, first, exchange)",False,False,False,False,False,False,False,False,False,False,False,False
"(random, first, insert)",False,False,False,False,False,False,False,False,False,False,False,False
"(random, first, transpose)",False,False,False,False,False,False,False,False,False,False,False,False
"(srz, best, exchange)",False,False,False,False,False,False,False,True,False,True,True,False
"(srz, best, insert)",False,False,False,False,False,False,True,False,False,True,False,False
"(srz, best, transpose)",False,False,False,False,False,False,False,False,False,False,False,True
"(srz, first, exchange)",False,False,False,False,False,False,True,True,False,False,True,False


In [284]:
exercise2 = merged[(merged.neighbourhood == "tei") | (merged.neighbourhood == "tie")]
print(len(exercise2))
exercise2.sample()

240


,init,instance,neighbourhood,pivot,score,time,best_solution,percentage_deviation
270,srz,100_20_13,tei,first,1749269,2.024878,1712830,2.127415


In [285]:
exercise2_groupby_algos = exercise2.groupby(['init', 'pivot', 'neighbourhood'])
exercise2_groupby_algos.mean()[['percentage_deviation']]

percentage_deviation
init   pivot neighbourhood                      
random first tei                        2.300083
             tie                        4.771551
srz    first tei                        2.550408
             tie                        2.745331